In [ ]:
import os
import copy
import time
import torch
import random
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from itertools import product
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

def fix_random(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(42) 

In [38]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [39]:
df = load_iris()

In [ ]:
# Dividi il dataset in feature (X) e target (y)
X = df.data
y = df.target

# Divisione del dataset: Train, Validation e Test set
# 1. Prima divisione: Separiamo il Test set (20%) dal resto (80%)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# 2. Seconda divisione: Dell'80% rimanente, prendiamo il 10% per il Validation e il resto per il Train
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)

# Stampiamo le dimensioni per verificare che tutto sia corretto
print("Campioni di Training (per allenare il modello): ", X_train.shape[0])
print("Campioni di Test (per valutare alla fine): ", X_test.shape[0])
print("Campioni di Validation (per ottimizzare durante il training): ", X_val.shape[0])

Number of train set:  108
Numebr of test set:  30
Number of validation set:  12


In [ ]:
# Conversione dei dati da NumPy arrays a PyTorch Tensors
# Le reti neurali in PyTorch lavorano con i "tensori" (matrici n-dimensionali che possono stare su GPU)

# Features (Input): Usiamo float32 perché sono numeri decimali continui
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

# Target (Output): Usiamo Long (interi) perché CrossEntropyLoss si aspetta 
# gli indici delle classi (0, 1, 2...), non numeri decimali.
y_train = torch.tensor(y_train, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# DataLoader: Creano dei "pacchetti" (batch) di dati.
# Invece di passare tutti i dati insieme, li passiamo a piccoli gruppi.
# Questo aiuta la rete ad imparare meglio e ad usare meno memoria.
# Per val e test usiamo un unico grande batch perché non dobbiamo fare backpropagation.
val_dataloader = DataLoader(TensorDataset(X_val, y_val), batch_size=y_val.shape[0])
test_dataloader = DataLoader(TensorDataset(X_test, y_test), batch_size=y_test.shape[0])

In [ ]:
def get_model(input_size, dept=3, hidden_size=64, dropout_prob=0.2):
    """
    Crea dinamicamente una rete neurale in base ai parametri passati.
    
    Args:
        input_size: numero di caratteristiche in ingresso (4 per Iris)
        dept (depth): profondità della rete, cioè quanti layer nascosti aggiungere
        hidden_size: quanti neuroni mettere in ogni layer nascosto
        dropout_prob: probabilità di spegnere neuroni (per evitare overfitting)
    """
    # 1. Primo layer: Connette l'input al primo strato nascosto
    model = [nn.Linear(input_size, hidden_size), nn.ReLU() ]
    
    # 2. Layer intermedi (Hidden): Vengono aggiunti in un ciclo in base a 'dept'
    for i in range(dept):
        model.append(nn.Linear(hidden_size, hidden_size)) # Connessione lineare
        model.append(nn.ReLU())                           # Attivazione non lineare
        model.append(nn.Dropout(dropout_prob))            # Regolarizzazione (Dropout)
        
    # 3. Output layer: Connette l'ultimo layer nascosto alle 3 classi finali
    model.append(nn.Linear(hidden_size, 3))
    
    # nn.Sequential unisce la lista di layer in un unico modello ordinato
    return nn.Sequential(*model)

In [ ]:
# --- GRID SEARCH SETUP ---
# Definiamo le liste di valori da provare per ogni iperparametro
hidden_size = [128, 256, 512]       # Neuroni per strato
dropout_prob = [0.2, 0.3, 0.4]      # Quanto "spegnere" la rete
dept = [3, 4, 5]                    # Quanti strati
epochs = 200                        # Durata training (fisso)
batch_size = [8,16,32]              # Dimensione pacchetto dati
learning_rate = [0.001, 0.01]       # Velocità di apprendimento

# itertools.product crea il Prodotto Cartesiano:
# Genera TUTTE le possibili combinazioni tra queste liste.
# Es: (128, 0.2, 3, 8, 0.001), (128, 0.2, 3, 8, 0.01), ...
params = product(hidden_size, dropout_prob, dept, batch_size, learning_rate)

# Calcoliamo quante combinazioni dovremo testare
combinations = len(hidden_size)*len(dropout_prob)*len(dept)*len(batch_size)*len(learning_rate)
print("Numero totale di configurazioni da testare: ", combinations)

Number of combinations:  162


In [ ]:
def train(model, train_dataloader, val_dataloader, device, hidden_size=3, dropout_prob=0.2, dept=2, epochs=100, batch_size=32, learning_rate=.001):
    """
    Esegue il training di un singolo modello.
    """
    # LOSS FUNCTION: Usiamo CrossEntropyLoss perché è un problema di Classificazione 
    criterion = nn.CrossEntropyLoss()
    
    # OPTIMIZER: Adam è l'algoritmo che aggiorna i pesi per minimizzare l'errore
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Liste per salvare lo storico degli errori
    train_loss = []
    val_loss = []

    # EARLY STOPPING:
    # Serve a fermare il training se il modello smette di migliorare sul validation set.
    best_model = None
    best_loss = np.inf   # Inizializziamo con infinito
    patience = 10        # Quante epoche aspettiamo se non migliora
    patience_counter = 0

    # --- CICLO DI TRAINING (EPOCHE) ---
    for epoch in range(epochs):
        epoch_start = time.time()
        epoch_loss = 0

        # Mettiamo il modello in modalità training (abilita Dropout, ecc.)
        model.train() 

        # Iteriamo sui batch di dati
        for x, y in train_dataloader:
            x, y = x.to(device), y.to(device) # Spostiamo dati su GPU/CPU
            
            optimizer.zero_grad()    # 1. Azzeriamo i gradienti precedenti
            y_pred = model(x)        # 2. Forward pass (previsione)
            loss = criterion(y_pred, y) # 3. Calcolo errore
            loss.backward()          # 4. Backward pass (calcolo gradienti)
            optimizer.step()         # 5. Aggiornamento pesi
            
            epoch_loss += loss.item()

        # Salviamo la loss media di training per questa epoca
        train_loss.append(epoch_loss / len(train_dataloader))

        # --- VALIDATION ---
        # Valutiamo come va il modello su dati che non ha usato per l'allenamento
        model.eval() # Modalità valutazione (disabilita Dropout)
        epoch_val_loss = 0
        with torch.no_grad(): # Disabilita calcolo gradienti (più veloce, meno memoria)
            for x, y in val_dataloader:
                x, y = x.to(device), y.to(device)
                y_pred = model(x)
                loss = criterion(y_pred, y)
                epoch_val_loss += loss.item()
        val_loss.append(epoch_val_loss / len(val_dataloader))

        # Stampa progressi ogni 10 epoche
        if (epoch+1) % 10 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Train loss: {train_loss[-1]:.4f}, Val loss: {val_loss[-1]:.4f}, Time: {time.time()-epoch_start:.2f}s')

        # --- EARLY STOPPING CHECK ---
        # Se l'errore di validazione è il più basso visto finora, salviamo questo modello
        if val_loss[-1] < best_loss:
            best_loss = val_loss[-1]
            best_model = copy.deepcopy(model) # Creiamo una copia esatta del modello attuale
            patience_counter = 0 # Resettiamo il contatore
        else:
            # Se non migliora, incrementiamo il contatore
            patience_counter += 1
            if patience_counter == patience:
                # Se abbiamo aspettato troppo ('patience' epoche), interrompiamo.
                print("Early stopping...") 
                break

    print("Training terminato in {} epoche. Miglior validation loss: {}".format(epoch+1, best_loss))

    # Restituiamo il miglior modello trovato (non necessariamente l'ultimo)
    return best_model, train_loss, val_loss

In [ ]:
def test_model(model, test_dataloader, device):
    """
    Valuta il modello finale sul Test Set per ottenere accuracy e predizioni.
    """
    model.eval() # Importante: modalità valutazione
    y_pred = []
    y_true = []
    
    with torch.no_grad():
        for x, y in test_dataloader:
            x , y = x.to(device), y.to(device)
            # model(x) restituisce 3 probabilità (logits).
            # torch.max(..., 1) ci dice qual è l'indice (0, 1 o 2) con il valore più alto.
            # quell'indice è la classe predetta.
            output = model(x)
            _, predicted = torch.max(output, 1) # Ottieni classe vincente
            
            y_pred.extend(predicted.cpu().tolist())
            y_true.extend(y.cpu().tolist())
            
    return y_pred, y_true

In [ ]:
# --- CICLO PRINCIPALE GRID SEARCH ---
# Qui proviamo ad allenare una rete diversa per ogni combinazione di iperparametri
best_model = None
best_accuracy = 0
best_config = None
iter_count = 0 

# Iteriamo su tutte le combinazioni generate da itertools.product
for hidden_size, dropout_prob, dept, batch_size, learning_rate in params:
    iter_count += 1
    print(f'\n--- Iterazione {iter_count}/{combinations} ---')
    print(f'Configurazione: Hidden={hidden_size}, Drop={dropout_prob}, Dept={dept}, Batch={batch_size}, LR={learning_rate}')

    # 1. Creiamo un NUOVO modello con questa specifica configurazione
    model = get_model(X_train.shape[1], dept=dept, hidden_size=hidden_size, dropout_prob=dropout_prob)
    
    # Creiamo il DataLoader specifico (perché il batch_size cambia)
    train_dataloader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)

    # Dizionario utile per salvare la configurazione corrente
    config = {
        'hidden_size': hidden_size,
        'dropout_prob': dropout_prob,
        'dept': dept,
        'batch_size': batch_size,
        'learning_rate': learning_rate
    }

    # 2. Alleniamo il modello e otteniamo la versione migliore (grazie all'early stopping)
    trained_model, train_loss, val_loss = train(model, train_dataloader, val_dataloader, device, **config)

    # 3. Testiamo il modello allenato sul Test Set
    y_pred, y_true = test_model(trained_model, test_dataloader, device)
    test_acc = accuracy_score(y_true, y_pred)
    
    print(f'Test Accuracy: {test_acc:.4f} (Migliore attuale: {best_accuracy:.4f})')

    # 4. Confronto: È questo il modello migliore visto finora?
    if test_acc > best_accuracy:
        best_accuracy = test_acc
        best_model = copy.deepcopy(trained_model) # Salviamo una copia
        best_config = config
        print("🏆 NUOVO RECORD!")



Iteration 1/162
hidden_size: 128, dropout_prob: 0.2, dept: 3, batch_size: 8, learning_rate: 0.001
Epoch 10/100, Train loss: 0.0596, Val loss: 0.2149, Time: 0.05s
Training in 19 epochs with best val loss: 0.16763661801815033
Test Accuracy: 0.8667 - Best Test Accuracy: 0.0000

Iteration 2/162
hidden_size: 128, dropout_prob: 0.2, dept: 3, batch_size: 8, learning_rate: 0.01
Epoch 10/100, Train loss: 0.3572, Val loss: 0.3462, Time: 0.05s
Training in 15 epochs with best val loss: 0.1263498067855835
Test Accuracy: 0.8667 - Best Test Accuracy: 0.8667

Iteration 3/162
hidden_size: 128, dropout_prob: 0.2, dept: 3, batch_size: 16, learning_rate: 0.001
Epoch 10/100, Train loss: 0.0568, Val loss: 0.3277, Time: 0.03s
Epoch 20/100, Train loss: 0.0452, Val loss: 0.5028, Time: 0.04s
Training in 26 epochs with best val loss: 0.17021971940994263
Test Accuracy: 0.8667 - Best Test Accuracy: 0.8667

Iteration 4/162
hidden_size: 128, dropout_prob: 0.2, dept: 3, batch_size: 16, learning_rate: 0.01
Epoch 10/1

In [47]:
print(f'Best config: {best_config}')
print(f'Best accuracy: {best_accuracy}')

Best config: {'hidden_size': 128, 'dropout_prob': 0.3, 'dept': 3, 'batch_size': 8, 'learning_rate': 0.01}
Best accuracy: 1.0


In [48]:
y_pred, y_true = test_model(best_model, test_dataloader, device)
acc = accuracy_score(y_true, y_pred)
print(f'Final Accuracy score: {acc}')

Final Accuracy score: 1.0
